<a href="https://colab.research.google.com/github/RomanTomz/RomeHeatmap/blob/master/Incidenti_Roma_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
#! pip install geojson
#! pip install geopandas

     |████████████████████████████████| 972kB 7.8MB/s 
     |████████████████████████████████| 10.9MB 22.2MB/s 
     |████████████████████████████████| 14.7MB 53.6MB/s 


In [153]:
import pandas as pd
import glob
import geojson
import plotly
import geopandas as gpd
import json
import plotly.graph_objects as go


In [117]:
df = pd.concat([pd.read_csv(f,  sep=';', encoding='latin1') for f in glob.glob('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/csv_incidenti_*.csv')])


In [118]:
df.drop_duplicates(subset='Protocollo', inplace=True)
df.DataOraIncidente = pd.to_datetime(df.DataOraIncidente)
df.sort_values('DataOraIncidente', inplace=True)

In [119]:
df

,Protocollo,Gruppo,DataOraIncidente,Localizzazione1,STRADA1,Localizzazione2,STRADA2,Strada02,Chilometrica,DaSpecificare,NaturaIncidente,particolaritastrade,TipoStrada,FondoStradale,Pavimentazione,Segnaletica,CondizioneAtmosferica,Traffico,Visibilita,Illuminazione,NUM_FERITI,NUM_RISERVATA,NUM_MORTI,NUM_ILLESI,Longitudine,Latitudine,Confermato,Progressivo,TipoVeicolo,StatoVeicolo,TipoPersona,Sesso,Tipolesione,Deceduto,DecedutoDopo,CinturaCascoUtilizzato,Airbag,Unnamed: 37,Longitude,Latitude
136,4160143,8,2019-01-01 01:10:00,Strada Urbana,VIA CASILINA,in prossimità,NaN,del civico n.,1765,NaN,Tamponamento,Rettilineo,Una carreggiata a doppio senso,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Intenso,Buona,Ore Diurne,1,0,0,2,12.6703,41.8640,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,F,Rifiuta cure immediate,0.0,NON DECEDUTO,Non accertato,Inesploso,NaN,NaN,NaN
75,4160005,1,2019-01-01 01:30:00,Strada Urbana,VIA DI MONTE TESTACCIO,in corrispondenza,NaN,del civico n.,39,NaN,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Buona,Sufficiente,0,0,0,3,12.4741,41.8757,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Illeso,0.0,NaN,Non accertato,Inesploso,NaN,NaN,NaN
3018,4181959,1,2019-01-01 02:00:00,Strada Urbana,PIAZZA DI S. CROCE IN GERUSALEMME,da specificare,NaN,NaN,NaN,impianto semaforico prima di Carlo Felice,Tamponamento,Rettilineo,Più di due carreggiate,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Sufficiente,Insufficiente,1,0,0,2,12.5147,41.8888,-1,1.0,Motociclo a solo,In marcia / fermata / arresto,Conducente,M,Rimandato,0.0,NON DECEDUTO,NaN,NaN,NaN,NaN,NaN
79,4160012,1,2019-01-01 02:00:00,Strada Urbana,LUNGOTEVERE TOR DI NONA,in prossimità,NaN,del palo luce n.,9,NaN,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Sufficiente,Sufficiente,0,0,0,9,12.4692,41.9015,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Illeso,0.0,NaN,Non accertato,NaN,NaN,NaN,NaN
24,4159601,20,2019-01-01 02:20:00,Strada Urbana,VIALE CORTINA D'AMPEZZO,in prossimità,NaN,del civico n.,198,NaN,Scontro frontale fra veicoli in marcia,Curva a visuale libera,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Scarso,Sufficiente,Sufficiente,4,0,0,0,12.4393,41.9465,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Ricoverato,0.0,NON DECEDUTO,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009,4733156,6,2019-12-31 20:05:00,Strada Urbana,VIA DI ACQUA BULLICANTE,all'intersezione con,VIA TEANO,VIA TEANO,NaN,NaN,Investimento di pedone,Intersezione stradale segnalata,Una carreggiata a doppio senso,Asciutto,Asfaltata,Orizzontale,Sereno,Normale,Sufficiente,Sufficiente,1,0,0,1,NaN,NaN,-1,NaN,NaN,NaN,Pedone,M,Rimandato,0.0,NON DECEDUTO,NaN,NaN,NaN,12.5421,41.8906
7049,4733260,1,2019-12-31 20:45:00,Strada Urbana,VIA MERULANA,in corrispondenza,NaN,del civico n.,90,NaN,Veicolo in marcia contro ostacolo fisso,Rettilineo,Una carreggiata a doppio senso,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Buona,Sufficiente,0,0,0,1,NaN,NaN,-1,1.0,Autovettura privata,Sosta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5022,41.8914
7059,4733275,18,2019-12-31 20:50:00,Strada Urbana,VIALE DI VALLE AURELIA,all'intersezione semaforizzata con,VIA BALDO DEGLI UBALDI,VIA BALDO DEGLI UBALDI,NaN,NaN,Scontro frontale/laterale SX fra veicoli in ma...,Intersezione semaforizzata,Due carreggiate,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Scarso,Sufficiente,Sufficiente,1,0,0,1,NaN,NaN,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Rifiuta cure immediate,0.0,NON DECEDUTO,NaN,NaN,NaN,12.4332,41.9171
7022,

In [120]:
viario = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/viario.csv', encoding='latin1', sep=';', header=None, usecols=[0,1,2])

In [121]:
viario.rename(columns={0:'municipio',1:'codice',2:'denominazione'}, inplace=True)

In [122]:
viario.denominazione = viario.denominazione.str.strip()

In [123]:
viario

,municipio,codice,denominazione
0,19,2357.0,VIA ABANO TERME
1,10,1126.0,VIA ABASCANTO
2,16,2530.0,VIA ABATE UGONE
3,20,2440.0,VIA ABBADIA SAN SALVATORE
4,19,2392.0,VIA ABBIATEGRASSO
...,...,...,...
30290,13,2048.0,VIA LUIGI CARDINI
30291,13,2048.0,VIA PAOLA ZANCANI MONTUORO
30292,8,627.0,VIA SAN PIETRO AVELLANA
30293,10,1191.0,VIA ROZZANO


In [124]:
df.STRADA1 = df.STRADA1.str.strip()

In [125]:
merged = pd.merge(df, viario, how='left', left_on='STRADA1', right_on='denominazione').drop_duplicates(subset='Protocollo')
merged.head(3)

,Protocollo,Gruppo,DataOraIncidente,Localizzazione1,STRADA1,Localizzazione2,STRADA2,Strada02,Chilometrica,DaSpecificare,NaturaIncidente,particolaritastrade,TipoStrada,FondoStradale,Pavimentazione,Segnaletica,CondizioneAtmosferica,Traffico,Visibilita,Illuminazione,NUM_FERITI,NUM_RISERVATA,NUM_MORTI,NUM_ILLESI,Longitudine,Latitudine,Confermato,Progressivo,TipoVeicolo,StatoVeicolo,TipoPersona,Sesso,Tipolesione,Deceduto,DecedutoDopo,CinturaCascoUtilizzato,Airbag,Unnamed: 37,Longitude,Latitude,municipio,codice,denominazione
0,4160143,8,2019-01-01 01:10:00,Strada Urbana,VIA CASILINA,in prossimità,NaN,del civico n.,1765,NaN,Tamponamento,Rettilineo,Una carreggiata a doppio senso,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Intenso,Buona,Ore Diurne,1,0,0,2,12.6703,41.8640,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,F,Rifiuta cure immediate,0.0,NON DECEDUTO,Non accertato,Inesploso,NaN,NaN,NaN,9,854.0,VIA CASILINA
60,4160005,1,2019-01-01 01:30:00,Strada Urbana,VIA DI MONTE TESTACCIO,in corrispondenza,NaN,del civico n.,39,NaN,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Buona,Sufficiente,0,0,0,3,12.4741,41.8757,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Illeso,0.0,NaN,Non accertato,Inesploso,NaN,NaN,NaN,1,1225.0,VIA DI MONTE TESTACCIO
61,4181959,1,2019-01-01 02:00:00,Strada Urbana,PIAZZA DI S. CROCE IN GERUSALEMME,da specificare,NaN,NaN,NaN,impianto semaforico prima di Carlo Felice,Tamponamento,Rettilineo,Più di due carreggiate,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Sufficiente,Insufficiente,1,0,0,2,12.5147,41.8888,-1,1.0,Motociclo a solo,In marcia / fermata / arresto,Conducente,M,Rimandato,0.0,NON DECEDUTO,NaN,NaN,NaN,NaN,NaN,1,16.0,PIAZZA DI S. CROCE IN GERUSALEMME


In [126]:
merged.columns

Index(['Protocollo', 'Gruppo', 'DataOraIncidente', 'Localizzazione1',
       'STRADA1', 'Localizzazione2', 'STRADA2', 'Strada02', 'Chilometrica',
       'DaSpecificare', 'NaturaIncidente', 'particolaritastrade', 'TipoStrada',
       'FondoStradale', 'Pavimentazione', 'Segnaletica',
       'CondizioneAtmosferica', 'Traffico', 'Visibilita', 'Illuminazione',
       'NUM_FERITI', 'NUM_RISERVATA', 'NUM_MORTI', 'NUM_ILLESI', 'Longitudine',
       'Latitudine', 'Confermato', 'Progressivo', 'TipoVeicolo',
       'StatoVeicolo', 'TipoPersona', 'Sesso', 'Tipolesione', 'Deceduto',
       'DecedutoDopo', 'CinturaCascoUtilizzato', 'Airbag', 'Unnamed: 37',
       'Longitude', 'Latitude', 'municipio', 'codice', 'denominazione'],
      dtype='object')

In [127]:
merged = merged[['DataOraIncidente','Localizzazione1','STRADA1','NaturaIncidente','particolaritastrade','TipoStrada','FondoStradale','Segnaletica','CondizioneAtmosferica','Traffico','Illuminazione','NUM_FERITI',
        'NUM_MORTI','NUM_ILLESI','Longitudine','Latitudine','TipoVeicolo','TipoPersona','Sesso','Tipolesione','DecedutoDopo','municipio']]

In [143]:
with open('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/ROMA_MUNICIPI.geojson', encoding='latin1') as f:
    municipi = json.load(f)

In [144]:
for feature in municipi['features']:
  municipi['id'] = feature['properties']['MUNICIPIO']

In [155]:
mun_feriti = merged.groupby('municipio')[['NUM_FERITI']].sum().reset_index(drop=False)

In [156]:
mun_feriti

,municipio,NUM_FERITI
0,1,1629
1,10,609
2,11,711
3,12,705
4,13,763
5,15,404
6,16,684
7,17,726
8,18,866
9,19,462


In [157]:
fig = go.Figure(go.Choroplethmapbox(geojson=municipi, locations=mun_feriti.municipio, z=mun_feriti.NUM_FERITI,
                                    colorscale="Reds", zmin=0, zmax=1500,
                                    marker_opacity=1, marker_line_width=0.6))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=4, mapbox_center = {"lat": 41.8719, "lon": 12.5674})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()